- cursor: fetch
- connect: db와 connect

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('existing_db.db')

In [3]:
cur = conn.cursor()

In [17]:
cur.executescript(
"""
    drop table artist;
    drop table genre;
    drop table albums;
    drop table track;
    create table artist (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name text
    );
    create table genre (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name text
    );
    create table albums (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title text,
        artist_id integer
    );
    create table track (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title text,
        length integer,
        rating integer,
        count integer,
        album_id integer,
        genre_id integer
    );
    insert into artist (name) values('Led Zepplin'); 
    insert into artist (name) values('AC/DC');
    
    insert into genre (name) values('Rock');
    insert into genre (name) values('Metal');
   
""")

In [23]:
conn.commit()
artist = cur.fetchall()


In [24]:
cur.executescript(
""" 
    insert into albums (title, artist_id) values ('Who Made Who', 2);
    insert into albums (title, artist_id) values ('IV', 1);
    
    insert into track (title, rating, length, count, album_id, genre_id) values ('Black Dog', 5, 297, 0,2,1);
    insert into track (title, rating, length, count, album_id, genre_id) values ('Stair Way', 5, 482, 0,2,1);
    insert into track (title, rating, length, count, album_id, genre_id) values ('About to Rock', 5, 313, 0,1,2);
    insert into track (title, rating, length, count, album_id, genre_id) values ('Who Made Who', 5, 207, 0,1,2);
""")

In [25]:
cur.execute("select * from track")

In [26]:
print(cur.fetchall())

[(1, 'Black Dog', 297, 5, 0, 2, 1), (2, 'Stair Way', 482, 5, 0, 2, 1), (3, 'About to Rock', 313, 5, 0, 1, 2), (4, 'Who Made Who', 207, 5, 0, 1, 2)]


# Join
- where절을 써도 되지만 join을 쓰는 것을 추천한다.
- where절을 쓰는 것이 편리하다.

In [36]:
cur.execute(
"""
    select albums.title, artist.name from albums join artist on albums.artist_id = artist.id
""")

In [37]:
print(cur.fetchall())

[('Who Made Who', 'AC/DC'), ('IV', 'Led Zepplin')]


In [38]:
cur.execute(
"""
    select track.title, genre.name from track join genre on track.genre_id = genre.id 
""")

In [39]:
print(cur.fetchall())

[('Black Dog', 'Rock'), ('Stair Way', 'Rock'), ('About to Rock', 'Metal'), ('Who Made Who', 'Metal')]


In [41]:
cur.execute("select albums.title, track.title from track join albums on track.album_id = albums.id")

In [42]:
print(cur.fetchall())

[('IV', 'Black Dog'), ('IV', 'Stair Way'), ('Who Made Who', 'About to Rock'), ('Who Made Who', 'Who Made Who')]


In [47]:
cur.execute(
"""
    select track.title, artist.name, albums.title, genre.name from track 
    join artist join albums join genre 
    on track.album_id = albums.id
    and track.genre_id = genre.id
    and albums.artist_id = artist.id
""")

# Cross Join

In [48]:
cur.execute("select * from artist, albums")
cur.execute("select * from artist join albums")
for row in cur.fetchall():
    print(row)

(1, 'Led Zepplin', 1, 'Who Made Who', 2)
(1, 'Led Zepplin', 2, 'IV', 1)
(2, 'AC/DC', 1, 'Who Made Who', 2)
(2, 'AC/DC', 2, 'IV', 1)


# Inner Join
- selecet + (선택 column)
- on + (condition)

In [51]:
cur.execute("select artist.name, albums.title from artist join albums on artist.id = albums.artist_id")

In [52]:
for row in cur.fetchall():
    print(row)

('AC/DC', 'Who Made Who')
('Led Zepplin', 'IV')


In [58]:
cur.execute(
    """
    select track.title, albums.title, genre.name, artist.name
    from track join albums join genre join artist
    on track.album_id = albums.id
    and track.genre_id = genre.id
    and albums.artist_id = artist.id
    """)
for row in cur.fetchall():
    print(row)

('Black Dog', 'IV', 'Rock', 'Led Zepplin')
('Stair Way', 'IV', 'Rock', 'Led Zepplin')
('About to Rock', 'Who Made Who', 'Metal', 'AC/DC')
('Who Made Who', 'Who Made Who', 'Metal', 'AC/DC')


# Dump(Backup)
- create dump file: __iterdump__
- open and read
- dump.db -> backup

In [60]:
with open('test.dump','w') as f:
    for line in conn.iterdump():
        f.write('%s\n'%line)

In [61]:
data = None
with open("test.dump","r") as f:
    data = f.read()

In [62]:
data

'BEGIN TRANSACTION;\nCREATE TABLE albums (\n        id INTEGER PRIMARY KEY AUTOINCREMENT,\n        title text,\n        artist_id integer\n    );\nINSERT INTO "albums" VALUES(1,\'Who Made Who\',2);\nINSERT INTO "albums" VALUES(2,\'IV\',1);\nCREATE TABLE artist (\n        id INTEGER PRIMARY KEY AUTOINCREMENT,\n        name text\n    );\nINSERT INTO "artist" VALUES(1,\'Led Zepplin\');\nINSERT INTO "artist" VALUES(2,\'AC/DC\');\nCREATE TABLE genre (\n        id INTEGER PRIMARY KEY AUTOINCREMENT,\n        name text\n    );\nINSERT INTO "genre" VALUES(1,\'Rock\');\nINSERT INTO "genre" VALUES(2,\'Metal\');\nDELETE FROM "sqlite_sequence";\nINSERT INTO "sqlite_sequence" VALUES(\'artist\',2);\nINSERT INTO "sqlite_sequence" VALUES(\'genre\',2);\nINSERT INTO "sqlite_sequence" VALUES(\'albums\',2);\nINSERT INTO "sqlite_sequence" VALUES(\'track\',4);\nCREATE TABLE track (\n        id INTEGER PRIMARY KEY AUTOINCREMENT,\n        title text,\n        length integer,\n        rating integer,\n        c

In [67]:
type(data) #string 이기 때문에 편집기로 열 수 있다.

str

In [63]:
conn = sqlite3.connect("dump.db")

In [64]:
cur = conn.cursor()

In [65]:
cur.executescript(data)

In [72]:
cur.execute("insert into track (title) values ('아무거나')")

In [73]:
cur.execute("select * from track")

In [74]:
print(cur.fetchall())

[(1, 'Black Dog', 297, 5, 0, 2, 1), (2, 'Stair Way', 482, 5, 0, 2, 1), (3, 'About to Rock', 313, 5, 0, 1, 2), (4, 'Who Made Who', 207, 5, 0, 1, 2), (5, '아무거나', None, None, None, None, None), (6, '아무거나', None, None, None, None, None)]


# Left Join
- from 기준 
- right이 없어도 상관없음 

In [75]:
cur.execute("select * from track left join albums on albums.id = track.album_id")

In [76]:
for row in cur.fetchall():
    print(row)

(1, 'Black Dog', 297, 5, 0, 2, 1, 2, 'IV', 1)
(2, 'Stair Way', 482, 5, 0, 2, 1, 2, 'IV', 1)
(3, 'About to Rock', 313, 5, 0, 1, 2, 1, 'Who Made Who', 2)
(4, 'Who Made Who', 207, 5, 0, 1, 2, 1, 'Who Made Who', 2)
(5, '아무거나', None, None, None, None, None, None, None, None)
(6, '아무거나', None, None, None, None, None, None, None, None)


In [77]:
cur.execute("select * from albums left join track on albums.id = track.album_id")

In [78]:
for row in cur.fetchall():
    print(row)

(1, 'Who Made Who', 2, 3, 'About to Rock', 313, 5, 0, 1, 2)
(1, 'Who Made Who', 2, 4, 'Who Made Who', 207, 5, 0, 1, 2)
(2, 'IV', 1, 1, 'Black Dog', 297, 5, 0, 2, 1)
(2, 'IV', 1, 2, 'Stair Way', 482, 5, 0, 2, 1)
